In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm
from util import *
from csae import *

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

## Use the distribution of signs so we don't have to try all possible signs

In [4]:
# Reproducibility
np.random.seed(42)

# Set actual amplitude and angle
a = 0.1
theta = np.arcsin(a)

# Set array parameters (see Eq. xxx)
narray = [2,2,2,2,2,2,2,2]

# Create ULA object. Requires array parameters narray and an optional C parameter 
# that defines how many samples to take.
ula_signal = TwoqULASignal(M=narray, C=5)
# Create ESPIRIT Object
esprit = ESPIRIT()


# This function "learns" the distribution of signs through simulation
heavy_signs = get_heavy_signs(ula_signal.depths, ula_signal.n_samples, len(narray)**2)


############ Simulation Code ##############
# This portion of code is only used for simulation purposes. If you wish to use this code
# with exprimental values, this will be replaced and one will set the measurement values
# directly using the ula_signal.set_measurements(measurements) method.

# Simulate a noisy set of measurements for the given angle theta. Simulator also returns the
# actual complext signal csignal that is not available from an actual experiment
csignal, measurements = simulate_signal(ula_signal.depths, ula_signal.n_samples, theta)
# Here we use the simulated complex signal to see how much better (or worse) we would have done
# if we had access to the actual complex signal and did not need to optimize the signs.
cR = ula_signal.get_cov_matrix_toeplitz(csignal)
theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
a_est1 = np.sin(theta_est1)
err1 = np.abs(a - np.sin(a_est1))
############ End Simulation Code ##############


# Set the measurement probabilities using an array of length depths that correspond to the 
# probability of measuring the |0,x> state in the state psi> = a|0,x> + sqrt{1-a^2}|1,x'>. 
# This corresponds to |a|^2. Here we use the simulated measurements, but this could be experimental
# measurements as well.
ula_signal.set_measurements(measurements)
# This function is a simple helper function that takes the ula_signal object and the learned 
# distribution of signs and estimates the amplitude a
a_est = estimate_amplitude(ula_signal, heavy_signs)
# Error is just the absolute difference
err = np.abs(a - a_est)


# Compute the total queries. We also add the 0 depth query estimate which is often neglected.
total_queries = np.sum(np.array(ula_signal.depths) * np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]

# Print summary output
print(f'query depths:            {ula_signal.depths}')
print(f'number of samples:       {ula_signal.n_samples}')
print(f'total number of queries: {total_queries}')
print(f'a exact:                 {a:0.9f}')
print(f'a estimated:             {a_est:0.9f}')
print(f'a estimated exact signs: {a_est1:0.9f}')
print(f'error:                   {err:0.3e}')
print(f'error exact signs:       {err1:0.3e}')


query depths:            [  0   1   2   4   8  16  32  64 128]
number of samples:       [90, 40, 35, 30, 25, 20, 15, 10, 5]
total number of queries: 2600
a exact:                 0.100000000
a estimated:             0.099796912
a estimated exact signs: 0.099898999
error:                   2.031e-04
error exact signs:       2.671e-04


## Let's do some statistics now

In [5]:
np.random.seed(42)



avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.1, 0.2, 0.3]
avals = [np.random.uniform(0.1, 0.9) for _ in range(15)]
# narray = [2]*5 + [3]
narray = [2]*4

# print('learning the distribution of signs...')
# heavy_signs = get_heavy_signs(narray, int(6*len(narray)))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()
# csignal, measurements = simulate_signal(ula_signal.depths, ula_signal.n_samples, theta)
# ula_signal.set_measurements(measurements)

# heavy_signs = get_heavy_signs(ula_signal, len(ula_signal.M)**2)
heavy_signs = get_heavy_signs(ula_signal.depths, ula_signal.n_samples, len(narray)**2)

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    disp=False
    # if j==4:
    #     disp=True
    for i in tqdm(range(num_mc)):
        csignal, measurements = simulate_signal(ula_signal.depths, ula_signal.n_samples, theta)
        ula_signal.set_measurements(measurements)
        
        res = csae_with_local_minimization(ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=disp)
        thetas[j][i] = res['theta_est']
        err = np.abs(np.sin(theta) - np.sin(thetas[j][i]))
        errors[j][i] = err

        cR = ula_signal.get_cov_matrix_toeplitz(csignal)
        theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
        err1 = np.abs(np.sin(theta) - np.sin(theta_est1))

        thetas1[j][i] = theta_est1
        errors1[j][i] = err1

        # thetas1[j][i] = res['theta_est1']
        # errors1[j][i] = res['error1']

    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 95):.3e}, {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

theta = 0.41111546403370536


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 8.396e-02, 15.112069502962964, 0.6716475334650206
theta = 1.0363905664439828


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.510e-02, 2.7187962089237976, 0.12083538706327988
theta = 0.7554209219922304


...

constant factors query and depth: 1.767e-02, 3.1806789581525385, 0.14136350925122393
theta = 0.2732593578259982


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 2.010e-02, 3.6180189579396407, 0.16080084257509514
theta = 0.24799415401854524


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 2.087e-02, 3.7570435958571786, 0.16697971537143017


In [6]:
perc = 68
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (68%) for 0.400: 9.705e-03, 1.747, 7.764e-02
constant factors query and depth (68%) for 0.861: 1.015e-02, 1.826, 8.117e-02
constant factors query and depth (68%) for 0.686: 8.145e-03, 1.466, 6.516e-02
constant factors query and depth (68%) for 0.579: 1.712e-02, 3.081, 1.370e-01
constant factors query and depth (68%) for 0.225: 9.906e-03, 1.783, 7.924e-02
constant factors query and depth (68%) for 0.225: 1.074e-02, 1.933, 8.589e-02
constant factors query and depth (68%) for 0.146: 1.041e-02, 1.873, 8.325e-02
constant factors query and depth (68%) for 0.793: 6.100e-03, 1.098, 4.880e-02
constant factors query and depth (68%) for 0.581: 2.015e-02, 3.627, 1.612e-01
constant factors query and depth (68%) for 0.666: 8.586e-03, 1.545, 6.869e-02
constant factors query and depth (68%) for 0.116: 1.407e-02, 2.533, 1.126e-01
constant factors query and depth (68%) for 0.876: 7.042e-03, 1.268, 5.634e-02
constant factors query and depth (68%) for 0.766: 9.408e-03, 1.6

In [7]:
perc = 68
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {np.percentile(errors1[j], perc):.3e}, {np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 1.663e-02, 2.993, 1.330e-01
constant factors query and depth with known signs for 0.861: 1.151e-02, 2.071, 9.206e-02
constant factors query and depth with known signs for 0.686: 1.403e-02, 2.525, 1.122e-01
constant factors query and depth with known signs for 0.579: 1.748e-02, 3.146, 1.398e-01
constant factors query and depth with known signs for 0.225: 1.477e-02, 2.659, 1.182e-01
constant factors query and depth with known signs for 0.225: 1.564e-02, 2.815, 1.251e-01
constant factors query and depth with known signs for 0.146: 1.772e-02, 3.190, 1.418e-01
constant factors query and depth with known signs for 0.793: 7.245e-03, 1.304, 5.796e-02
constant factors query and depth with known signs for 0.581: 2.130e-02, 3.834, 1.704e-01
constant factors query and depth with known signs for 0.666: 1.036e-02, 1.866, 8.292e-02
constant factors query and depth with known signs for 0.116: 2.157e-02, 3.883, 1.726e-01
constant factors quer

In [8]:
perc = 95
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (95%) for 0.400: 8.396e-02, 15.112, 6.716e-01
constant factors query and depth (95%) for 0.861: 1.510e-02, 2.719, 1.208e-01
constant factors query and depth (95%) for 0.686: 1.470e-02, 2.646, 1.176e-01
constant factors query and depth (95%) for 0.579: 3.326e-02, 5.987, 2.661e-01
constant factors query and depth (95%) for 0.225: 2.035e-02, 3.663, 1.628e-01
constant factors query and depth (95%) for 0.225: 2.524e-02, 4.543, 2.019e-01
constant factors query and depth (95%) for 0.146: 4.652e-02, 8.373, 3.722e-01
constant factors query and depth (95%) for 0.793: 1.203e-02, 2.165, 9.623e-02
constant factors query and depth (95%) for 0.581: 2.791e-02, 5.023, 2.233e-01
constant factors query and depth (95%) for 0.666: 1.345e-02, 2.421, 1.076e-01
constant factors query and depth (95%) for 0.116: 1.834e-02, 3.301, 1.467e-01
constant factors query and depth (95%) for 0.876: 2.096e-02, 3.773, 1.677e-01
constant factors query and depth (95%) for 0.766: 1.767e-02, 3.

In [9]:
perc = 95
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {np.percentile(errors1[j], perc):.3e}, {np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 4.728e-02, 8.511, 7.565e-01
constant factors query and depth with known signs for 0.861: 1.517e-02, 2.731, 2.428e-01
constant factors query and depth with known signs for 0.686: 1.789e-02, 3.220, 2.862e-01
constant factors query and depth with known signs for 0.579: 2.924e-02, 5.263, 4.678e-01
constant factors query and depth with known signs for 0.225: 2.721e-02, 4.899, 4.354e-01
constant factors query and depth with known signs for 0.225: 2.771e-02, 4.988, 4.434e-01
constant factors query and depth with known signs for 0.146: 4.938e-02, 8.888, 7.901e-01
constant factors query and depth with known signs for 0.793: 1.311e-02, 2.360, 2.098e-01
constant factors query and depth with known signs for 0.581: 2.740e-02, 4.932, 4.384e-01
constant factors query and depth with known signs for 0.666: 1.456e-02, 2.621, 2.330e-01
constant factors query and depth with known signs for 0.116: 2.859e-02, 5.147, 4.575e-01
constant factors quer

In [10]:
perc = 99
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (99%) for 0.400: 1.394e-01, 25.099, 1.115e+00
constant factors query and depth (99%) for 0.861: 2.438e-02, 4.388, 1.950e-01
constant factors query and depth (99%) for 0.686: 1.800e-02, 3.241, 1.440e-01
constant factors query and depth (99%) for 0.579: 4.451e-02, 8.012, 3.561e-01
constant factors query and depth (99%) for 0.225: 6.880e-02, 12.383, 5.504e-01
constant factors query and depth (99%) for 0.225: 7.244e-02, 13.038, 5.795e-01
constant factors query and depth (99%) for 0.146: 5.447e-02, 9.805, 4.358e-01
constant factors query and depth (99%) for 0.793: 2.219e-02, 3.994, 1.775e-01
constant factors query and depth (99%) for 0.581: 5.537e-02, 9.967, 4.430e-01
constant factors query and depth (99%) for 0.666: 1.928e-02, 3.470, 1.542e-01
constant factors query and depth (99%) for 0.116: 2.446e-02, 4.403, 1.957e-01
constant factors query and depth (99%) for 0.876: 2.874e-02, 5.174, 2.299e-01
constant factors query and depth (99%) for 0.766: 3.173e-02, 

In [11]:
perc = 99
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {np.percentile(errors1[j], perc):.3e}, {np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 4.973e-02, 8.951, 7.956e-01
constant factors query and depth with known signs for 0.861: 1.719e-02, 3.093, 2.750e-01
constant factors query and depth with known signs for 0.686: 1.867e-02, 3.361, 2.988e-01
constant factors query and depth with known signs for 0.579: 3.271e-02, 5.887, 5.233e-01
constant factors query and depth with known signs for 0.225: 4.474e-02, 8.053, 7.158e-01
constant factors query and depth with known signs for 0.225: 5.487e-02, 9.876, 8.779e-01
constant factors query and depth with known signs for 0.146: 5.383e-02, 9.689, 8.612e-01
constant factors query and depth with known signs for 0.793: 1.688e-02, 3.038, 2.701e-01
constant factors query and depth with known signs for 0.581: 3.201e-02, 5.762, 5.122e-01
constant factors query and depth with known signs for 0.666: 1.765e-02, 3.177, 2.824e-01
constant factors query and depth with known signs for 0.116: 3.156e-02, 5.681, 5.049e-01
constant factors quer